In [16]:
import json
from web3 import Web3
from web3.middleware import geth_poa_middleware
import pickle
import requests
import pandas as pd
from datetime import datetime
import numpy as np
import math

In [17]:
web3 = Web3(Web3.HTTPProvider("https://bsc-dataseed.binance.org/"))
web3.middleware_onion.inject(geth_poa_middleware, layer=0)
web3.isConnected()

True

In [18]:
# Address = token contract
apikey = "GKDPH6RRX5YSH1TYPMNHNVKGMB6I9V3I8T"
address = "0xF700D4c708C2be1463E355F337603183D20E0808"

# Methods
approval_method = "0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925"
transfer_method = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"
stake_method = "0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c"


# Set block
currentblock = web3.eth.blockNumber
endblock = currentblock

# Set topic0
topic0 = approval_method

# Current block
currentblock = web3.eth.blockNumber
endblock = currentblock

# Otros contratos
claim_contract = "0x739e6e290f9aa6ef3a929950aa47795b1d187229"
stake_contract = "0xdf5c4c1b98a4d045573ab2005d77e87daa58c853"


# Liquidity Pool
pcs = "0x00000000000000000000000072121d60b0e2f01c0fb7fe32ca24021b42165a40"
pcs_address = "0x72121d60b0e2F01c0FB7FE32cA24021b42165A40"

ape = "0x000000000000000000000000cC3A3Bc1d76Df321f94716E88224638C439267aa"
ape_address = "0xcC3A3Bc1d76Df321f94716E88224638C439267aa"

biswap = "0x000000000000000000000000B4c88543Bd95617Fb61F3d031B8E8E19aF33201E"
biswap_address = "0xB4c88543Bd95617Fb61F3d031B8E8E19aF33201E"

# Set block
currentblock = web3.eth.blockNumber
endblock = currentblock

# Set topic0
topic0 = approval_method

# Wallets to check
check_wallets = [
    address,
    claim_contract,
    stake_contract,
    pcs_address,
    ape_address,
    biswap_address,
]

In [19]:
# Get ABI

url = "https://api.bscscan.com/api"
params = {
    "module": "contract",
    "action": "getabi",
    "address": address,
    "apikey": apikey,
}

response = requests.get(url, params=params)

if response.status_code == 200:
    abi = response.json()["result"]
    print("Abi recibida correctamente")
else:
    print("Error en la solicitud")

Abi recibida correctamente


In [20]:
w3address = Web3.toChecksumAddress(address.lower())
contract = web3.eth.contract(address=w3address, abi=abi)

token = contract.functions.name().call()
symbol = contract.functions.symbol().call()

print(f"Abi del token: {token}, con simbolo: {symbol}, recibida correctamente")

Abi del token: Galactic Quadrant, con simbolo: GQ, recibida correctamente


In [21]:
# Get "Internal Transactions" by Block Range
def get_internal_transactions(endblock, numberofblocks):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "account",
        "action": "txlistinternal",
        "startblock": endblock - numberofblocks,
        "endblock": endblock,
        "page": 1,
        "offset": 10,
        "sort": "asc",
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        transactions = response.json()["result"]
        return transactions
    else:
        return "Error en la solicitud"


# Get Logs by Address sin topic0
def get_logs_loop(address, endblock, blocks):
    if blocks % 1000 != 0:
        raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks / 1000)):
        logs = get_logs1000(address, endblock - loop * 1000)
        total_logs = total_logs + logs
    return total_logs


def get_logs1000(address, endblock):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": endblock - 1000,
        "toBlock": endblock,
        "address": address,
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"


# Get Logs by Address
def get_logstopic_loop(address, endblock, blocks, topic0):
    if blocks % 1000 != 0:
        raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks / 1000)):
        logs = get_logs1000(address, endblock - loop * 1000)
        total_logs = total_logs + logs
    return total_logs


def get_logs1000topic(address, endblock, topic0):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": endblock - 1000,
        "toBlock": endblock,
        "topic0": topic0,
        "address": address,
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"


# Get Logs by Address and From
def get_logstopic0_1_loop(address, endblock, blocks, topic0, topic1):
    if blocks % 1000 != 0:
        raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks / 1000)):
        logs = get_logs1000(address, endblock - loop * 1000)
        total_logs = total_logs + logs
    return total_logs


def get_logs1000topic0_1(address, endblock, topic0, topic1):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": endblock - 1000,
        "toBlock": endblock,
        "topic0": topic0,
        "topic0_1": "and",
        "topic1": topic1,
        "address": address,
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"


# Get a list of 'BEP-20 Token Transfer Events' by wallet
def get_token_transfers(wallet, address):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "account",
        "action": "tokentx",
        "contractaddress": address,
        "address": wallet,
        "page": 1,
        "offset": 1000,
        "startblock": 0,
        "endblock": 999999999,
        "sort": "asc",
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        transactions = response.json()["result"]
        return transactions
    else:
        return "Error in the request"


# Remove known wallets
def delete_wallet_from_df(df, check_wallets):
    for wallet_address in check_wallets:
        result = df["wallets"].isin([wallet_address.lower()]).any()
        print(f"Esta {wallet_address} en las wallets? {result}")
        if result:
            df = df.query('wallets != "{}"'.format(wallet_address.lower()))
    return df

In [22]:
## Extractor de Events/topics0
total_logs = get_logs_loop(address, endblock, 100000)


# Save hash in a list
total_hash = []
for i in range(len(total_logs)):
    event_hash = total_logs[i]["topics"][0]
    total_hash.append(event_hash)

unique_event_hash = list(set(total_hash))


# Cantidad de logs extraidos
print(
    f"Se han obtenido {len(total_logs)} logs, en los cuales hay {len(unique_event_hash)} event hash unicos."
)
print(unique_event_hash)


# Guarda los nuevos event hash
logs_unique = []
for value in unique_event_hash:
    if value not in logs_unique:
        logs_unique.append(value)

print(
    f"Actualmente hay almacenados {len(logs_unique)} event hash unicos que utilizaremos para reconocer los diferentes methods de topic0"
)

Se han obtenido 15916 logs, en los cuales hay 3 event hash unicos.
['0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef', '0x9d228d69b5fdb8d273a2336f8fb8612d039631024ea9bf09c424a9503aa078f0', '0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925']
Actualmente hay almacenados 3 event hash unicos que utilizaremos para reconocer los diferentes methods de topic0


In [23]:
# Un dia son aprox 28200 bloques, queremos calcular el finde pasado -5/-7 dias
currentblock = web3.eth.blockNumber
endblock = currentblock - (28200 * 4)
blocks = 28200 * 2.5
blocks = math.ceil(blocks / 1000.0) * 1000  # Multiplos de 1000

# Get data
logs_weekend = get_logs_loop(address, endblock, blocks)
token_tx = pd.DataFrame(logs_weekend)
token_tx.head()

,address,topics,data,blockNumber,blockHash,timeStamp,gasPrice,gasUsed,logIndex,transactionHash,transactionIndex
0,0xf700d4c708c2be1463e355f337603183d20e0808,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,0x00000000000000000000000000000000000000000000...,0x1813e0d,0xa5a71756b28324a0bcf292c40d7decb1a44a1f2df191...,0x63d85641,0x12a05f200,0x1c472,0x52,0xdbca23a8f2839ac50bfca6d1c6bfd34af785d125baff...,0x12
1,0xf700d4c708c2be1463e355f337603183d20e0808,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,0x00000000000000000000000000000000000000000000...,0x1813e0d,0xa5a71756b28324a0bcf292c40d7decb1a44a1f2df191...,0x63d85641,0x12a05f200,0x1c472,0x53,0xdbca23a8f2839ac50bfca6d1c6bfd34af785d125baff...,0x12
2,0xf700d4c708c2be1463e355f337603183d20e0808,[0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b...,0xffffffffffffffffffffffffffffffffffffffffffff...,0x1813e0d,0xa5a71756b28324a0bcf292c40d7decb1a44a1f2df191...,0x63d85641,0x12a05f200,0x1c472,0x54,0xdbca23a8f2839ac50bfca6d1c6bfd34af785d125baff...,0x12
3,0xf700d4c708c2be1463e355f337603183d20e0808,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,0x00000000000000000000000000000000000000000000...,0x1813e0e,0x40b1da977a73a2a1b9be19dd43b02b9bd8601478497e...,0x63d85644,0x12a05f200,0xb297,0x44,0x24a182c7dfcc6f736699fdda3a238aaa474cb0a9a5c9...,0x1c
4,0xf700d4c708c2be1463e355f337603183d20e0808,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,0x00000000000000000000000000000000000000000000...,0x1813e1d,0x99585e885c124345dc8e7483e6de469fd1bf2dbb6930...,0x63d85671,0x12a05f200,0xc2ff,0xc9,0x44b91bf00eecc2decc8b49ca754072572687a0aaf131...,0x33


In [24]:
# TOKEN LOG CLEANING

# Data Clean
token_tx["timeStamp"] = token_tx["timeStamp"].apply(lambda x: int(x, 16))
token_tx["timeStamp"] = token_tx["timeStamp"].apply(lambda x: datetime.fromtimestamp(x))

df_topics = pd.DataFrame(token_tx["topics"].tolist(), columns=["method", "from", "to"])
token_tx = pd.concat([token_tx, df_topics], axis=1)

token_tx = token_tx.drop(
    columns=[
        "topics",
        "address",
        "data",
        "blockNumber",
        "blockHash",
        "gasPrice",
        "gasUsed",
        "logIndex",
        "gasUsed",
        "transactionIndex",
    ],
    axis=1,
)

token_tx.sort_values(by="timeStamp", ascending=True, inplace=True)

# Filtrar la fecha
token_tx["intime"] = 0
for index, row in token_tx.iterrows():
    if row["timeStamp"].dayofyear < 31 and row["timeStamp"].dayofyear > 28:
        token_tx.at[index, "intime"] = 1

token_tx.head()

token_tx["intime"] = token_tx["intime"].astype(bool)
token_tx = token_tx[token_tx["intime"] == 1]

token_tx = token_tx.drop(
    columns=["intime"],
    axis=1,
)

# Standarizar address, from & to
token_tx["to"] = "0x" + token_tx["to"].str[26:]
token_tx["from"] = "0x" + token_tx["from"].str[26:]

# Filtrar solo transferencias
token_tx = token_tx[token_tx["method"] == transfer_method]

In [25]:
token_tx.head()

,timeStamp,transactionHash,method,from,to
9093,2023-01-29 00:04:18,0x6b7ca16a1dc279b6540ba156557714ce4e13ee986522...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0xb88d09175676913d69924acac17ec69037f1551e,0x4571d47e9ed7d838e9af02ef7ae3f54c56650a40
9097,2023-01-29 00:05:18,0x5cab2d07852db057169120f1f42ad0e5ef912ab0bd06...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0xef713415d4c058cb3cd5224174e28c9384d08e21,0xb4c88543bd95617fb61f3d031b8e8e19af33201e
9095,2023-01-29 00:05:18,0xe7fc044ab8530a0c6ede611ad2b4fa499db181c57c69...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0xb4c88543bd95617fb61f3d031b8e8e19af33201e
9094,2023-01-29 00:05:18,0x0cc6b78ba8e98cfc7b562b215caf886673f75adc6df7...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0xb4c88543bd95617fb61f3d031b8e8e19af33201e,0xf1ec5c64da312f96aef027c3dd26880b4e1a59b6
9096,2023-01-29 00:05:18,0x5cab2d07852db057169120f1f42ad0e5ef912ab0bd06...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0xcc3a3bc1d76df321f94716e88224638c439267aa,0xef713415d4c058cb3cd5224174e28c9384d08e21


In [26]:
token_tx["from"].value_counts()

0x72121d60b0e2f01c0fb7fe32ca24021b42165a40    1953
0xcc3a3bc1d76df321f94716e88224638c439267aa     346
0xb88d09175676913d69924acac17ec69037f1551e     279
0xb4c88543bd95617fb61f3d031b8e8e19af33201e     263
0xefdca55e4bce6c1d535cb2d0687b5567eef2ae83     223
                                              ... 
0xcb0d4e021b66cbe874629cdcb00737bd2fb3bbab       1
0x568ad3c51e1b09bcd4d1a698bfb5756ae745afbf       1
0x44fda19233f41c1bc282be5af4602b4145093e5b       1
0xd4be109c7c7a83c91411df8e33ba5388ef766cc5       1
0xd3ac3ff660fbdc812fa929a0d649627d77439d26       1
Name: from, Length: 992, dtype: int64

In [27]:
wallets = token_tx["from"].unique()
wallets = pd.DataFrame(wallets, columns=["wallets"])
wallets

,wallets
0,0xb88d09175676913d69924acac17ec69037f1551e
1,0xef713415d4c058cb3cd5224174e28c9384d08e21
2,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40
3,0xb4c88543bd95617fb61f3d031b8e8e19af33201e
4,0xcc3a3bc1d76df321f94716e88224638c439267aa
...,...
987,0xdbc14d3dede38d400b79dcb408ad5bf35acf06bd
988,0x8495773eb82f656fc4e5729dc37934e80f212d29
989,0x3b0114849d72ac001a0009062d091d69797325f5
990,0x6d9f2a9ffe25442674302cbbae41564b40d9e85c


In [28]:
wallets = delete_wallet_from_df(wallets, check_wallets)

Esta 0xF700D4c708C2be1463E355F337603183D20E0808 en las wallets? False
Esta 0x739e6e290f9aa6ef3a929950aa47795b1d187229 en las wallets? True
Esta 0xdf5c4c1b98a4d045573ab2005d77e87daa58c853 en las wallets? True
Esta 0x72121d60b0e2F01c0FB7FE32cA24021b42165A40 en las wallets? True
Esta 0xcC3A3Bc1d76Df321f94716E88224638C439267aa en las wallets? True
Esta 0xB4c88543Bd95617Fb61F3d031B8E8E19aF33201E en las wallets? True


In [29]:
wallets = delete_wallet_from_df(wallets, check_wallets)

Esta 0xF700D4c708C2be1463E355F337603183D20E0808 en las wallets? False
Esta 0x739e6e290f9aa6ef3a929950aa47795b1d187229 en las wallets? False
Esta 0xdf5c4c1b98a4d045573ab2005d77e87daa58c853 en las wallets? False
Esta 0x72121d60b0e2F01c0FB7FE32cA24021b42165A40 en las wallets? False
Esta 0xcC3A3Bc1d76Df321f94716E88224638C439267aa en las wallets? False
Esta 0xB4c88543Bd95617Fb61F3d031B8E8E19aF33201E en las wallets? False


In [30]:
wallets.to_csv("gq_bought_weekend.csv", index=False)